Data University

Kafka Assignment

Author: Tryambak Kaushik

Q: The Tracking Bitcoin Prices example covered during the slides uses Kafka Streaming for implementing the logic.

For this assignment redo the example above using Spark Streaming instead of Kafka Streaming

In [1]:
import socket
import sys
import requests as req
import requests_oauthlib
import json
import time

In [2]:
ticker_url="https://blockchain.info/ticker"
sleep_time=2

In [3]:
TCP_IP = "localhost"
TCP_PORT = 9997
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print "\nConnected to client at ", addr,"\n"
value_serializer=lambda x: json.dumps(x).encode('utf-8')

while (True):
    time.sleep(sleep_time)
    try:
        resp = req.get(ticker_url)
        json_data = json.loads(resp.text)
        print('USD Data', json_data['USD'])
        conn.send(value_serializer(json_data) + "\n")
    except:
        e = sys.exc_info()[0]
        print("Error4: %s" % e)

Waiting for TCP connection...
Connected to client at  ('127.0.0.1', 45231)
{u'sell': 9421.33, u'last': 9421.33, u'symbol': u'$', u'buy': 9421.33, u'15m': 9421.33}
{u'sell': 9421.33, u'last': 9421.33, u'symbol': u'$', u'buy': 9421.33, u'15m': 9421.33}
{u'sell': 9421.33, u'last': 9421.33, u'symbol': u'$', u'buy': 9421.33, u'15m': 9421.33}
{u'sell': 9421.33, u'last': 9421.33, u'symbol': u'$', u'buy': 9421.33, u'15m': 9421.33}
{u'sell': 9421.33, u'last': 9421.33, u'symbol': u'$', u'buy': 9421.33, u'15m': 9421.33}
{u'sell': 9419.02, u'last': 9419.02, u'symbol': u'$', u'buy': 9419.02, u'15m': 9419.02}
{u'sell': 9419.02, u'last': 9419.02, u'symbol': u'$', u'buy': 9419.02, u'15m': 9419.02}
{u'sell': 9419.02, u'last': 9419.02, u'symbol': u'$', u'buy': 9419.02, u'15m': 9419.02}
{u'sell': 9419.02, u'last': 9419.02, u'symbol': u'$', u'buy': 9419.02, u'15m': 9419.02}
{u'sell': 9419.02, u'last': 9419.02, u'symbol': u'$', u'buy': 9419.02, u'15m': 9419.02}
{u'sell': 9419.02, u'last': 9419.02, u'symbol

In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
# Create a local StreamingContext with two working thread and batch interval of 1 second
from pyspark import SparkContext, SQLContext, Row
from pyspark.streaming import StreamingContext

In [4]:
import json

In [5]:
spc = SparkContext("local[2]", "LocalWordCount")
stc = StreamingContext(spc, 10)
stc.checkpoint("checkpoint")

sqlContext = SQLContext(spc)

In [6]:
if 'prev_price_out' not in globals():
    globals()['prev_price_out']=0
    
    
def get_data(time, message):
    
    print("\n========= %s =========" % str(time))

    prev_price = globals()['prev_price_out']
    
    load_rdd0 = message.map(lambda x: json.loads(x))
    load_rdd1 = load_rdd0.map(lambda x: x['USD']['sell'])
    
    if len (load_rdd1.collect()) > 0 :
    
        new_price = float(load_rdd1.collect()[-1])
        strike_price=round(prev_price - 0.0005*(prev_price), 2)

        print("Previous price  : " + str(prev_price))
        print("New price       : " + str(new_price))
        print("Strike Price    : " + str(strike_price))

        if ( new_price < strike_price ):
            print "Price is less than previous values by 0.05%. Maybe time to initiate a BUY."
        else:
            print "Price is not less than previous values by 0.05%"

        globals()['prev_price_out']=new_price
        print("-----------------------------------------")
    
    return

lines = stc.socketTextStream("localhost", 9997)
lines.foreachRDD(get_data)

In [7]:
stc.start()             # Start the computation
stc.awaitTermination()  # Wait for the compute

========= 2020-06-17 21:01:50 =========
('prev_price=', 0)
Previous price  : 0
New price       : 9421.33
Strike Price    : 0.0
Price is not less than previous values by 0.05%
-----------------------------------------
========= 2020-06-17 21:02:00 =========
('prev_price=', 9421.33)
Previous price  : 9421.33
New price       : 9421.33
Strike Price    : 9416.62
Price is not less than previous values by 0.05%
-----------------------------------------
========= 2020-06-17 21:02:10 =========
('prev_price=', 9421.33)
Previous price  : 9421.33
New price       : 9419.02
Strike Price    : 9416.62
Price is not less than previous values by 0.05%
-----------------------------------------
========= 2020-06-17 21:02:20 =========
('prev_price=', 9419.02)
Previous price  : 9419.02
New price       : 9419.02
Strike Price    : 9414.31
Price is not less than previous values by 0.05%
-----------------------------------------
========= 2020-06-17 21:02:30 =========
('prev_price=', 9419.02)
Previous price  : 9

In [ ]:
stc.stop()